# MVD 5. cvičení

## 1. část - TF-IDF s word embeddingy

V minulém cvičení bylo za úkol implementovat TF-IDF algoritmus nad datasetem z Kagglu. Dnešní cvičení je rozšířením této úlohy s použitím word embeddingů. Lze použít předtrénované GloVe embeddingy ze 3. cvičení, nebo si v případě zájmu můžete vyzkoušet práci s Word2Vec od Googlu (najdete [zde](https://code.google.com/archive/p/word2vec/)).

Cvičení by mělo obsahovat následující části:
- Načtení článků a embeddingů
- Výpočet document vektorů pomocí TF-IDF a word embeddingů 
    - Pro výpočet TF-IDF využijte [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) z knihovny sklearn
    - Vážený průměr GloVe / Word2Vec vektorů

<center>
$
doc\_vector = \frac{1}{|d|} \sum\limits_{w \in d} TF\_IDF(w) glove(w)
$
</center>

- Dotaz bude transformován stejně jako dokument

- Výpočet relevance pomocí kosinové podobnosti
<center>
$
score(q,d) = cos\_sim(query\_vector, doc\_vector)
$
</center>

### Načtení článků

In [32]:
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import TfidfVectorizer

In [33]:
csv_filename = "/Users/kirillefremov/development/PycharmProjects/TUL-MVD-2024-25/cv04/articles.csv"
titles = []
texts = []

with open(csv_filename, 'r') as file:
    articles = csv.reader(file)
    print(next(articles.__iter__()), end="\n\n")  # Printing the columns names
    
    for line in articles:
        titles.append(line[-2])
        texts.append(line[-1])

print(titles[0], end="\n\n")
print(texts[0])
print(f"Titles number: {len(titles)}, Texts number: {len(texts)}")

['author', 'claps', 'reading_time', 'link', 'title', 'text']

Chatbots were the next big thing: what happened? – The Startup – Medium

Oh, how the headlines blared:
Chatbots were The Next Big Thing.
Our hopes were sky high. Bright-eyed and bushy-tailed, the industry was ripe for a new era of innovation: it was time to start socializing with machines.
And why wouldn’t they be? All the road signs pointed towards insane success.
At the Mobile World Congress 2017, chatbots were the main headliners. The conference organizers cited an ‘overwhelming acceptance at the event of the inevitable shift of focus for brands and corporates to chatbots’.
In fact, the only significant question around chatbots was who would monopolize the field, not whether chatbots would take off in the first place:
One year on, we have an answer to that question.
No.
Because there isn’t even an ecosystem for a platform to dominate.
Chatbots weren’t the first technological development to be talked up in grandiose terms 

### Načtení embeddingů

In [34]:
GLOVE_SIZE = 50

path = "/Users/kirillefremov/development/PycharmProjects/TUL-MVD-2024-25/cv03/"
filename = 'glove.6B/glove.6B.50d.txt'
# filename = 'glove.6B/glove.6B.100d.txt'
# filename = 'glove.6B/glove.6B.200d.txt'
# filename = 'glove.6B/glove.6B.300d.txt'

with open(path + filename, 'r') as f:
    glove = {}
    for line in f:
        line = line.strip().split()
        word = line[0]
        vec = np.array(line[1:], dtype="float32")
        glove[word] = vec
        
print(list(glove.keys())[:5])

['the', ',', '.', 'of', 'to']


### TF-IDF + Word2Vec a vytvoření doc vektorů

In [35]:
def calculate_doc_vector(doc_id, tfidf_matrix, word_to_index, glove, glove_size=GLOVE_SIZE):
    """
    Weighted average of GloVe/Word2Vec vectors for current document.

    Params:
        doc_id: int / None -- id for current document (index in list of docs). None is given while there is only one document in the doc list.
        tfidf_matrix: sklearn matrix -- tfidf_matrix for all documents and words/
        word_to_index: dict[str]: int -- dictionary of indeces for every word in tfidf matrix columns.
        glove: dict[str]: int -- dictionary of vector representation for dataset words.
        glove_size: int -- size of glove vectors.

    Return:
        doc_vector.
    """

    if doc_id:
        words_in_doc = tfidf_matrix[doc_id].toarray().flatten()
    else:
        words_in_doc = tfidf_matrix.toarray().flatten()

    doc_vector = np.zeros(glove_size)

    word_count = np.sum(words_in_doc > 0)

    for word, index in word_to_index.items():
        tfidf_value = words_in_doc[index]  # TF-IDF
        word_vector = glove.get(word, np.zeros(glove_size)) 
        doc_vector += tfidf_value * word_vector
        
    return doc_vector / word_count

In [36]:
vectorizer = TfidfVectorizer()

tfidf_matrix = vectorizer.fit_transform(texts)
word_to_index = vectorizer.vocabulary_
print(vectorizer.get_feature_names_out()[1000:1005])

# with open('output.txt', 'w', encoding='utf-8') as f:
#     for v in vectorizer.get_feature_names_out():
#         f.write(f"{v}\n")

print(tfidf_matrix.shape)
print(word_to_index)

['advancement' 'advancements' 'advances' 'advancing' 'advantage']
(337, 20396)
{'oh': 11972, 'how': 8452, 'the': 17240, 'headlines': 8139, 'blared': 2434, 'chatbots': 3219, 'were': 18807, 'next': 11623, 'big': 2349, 'thing': 17290, 'our': 12188, 'hopes': 8399, 'sky': 15766, 'high': 8262, 'bright': 2690, 'eyed': 6640, 'and': 1379, 'bushy': 2805, 'tailed': 16967, 'industry': 8906, 'was': 18692, 'ripe': 14758, 'for': 7118, 'new': 11602, 'era': 6276, 'of': 11944, 'innovation': 9037, 'it': 9433, 'time': 17403, 'to': 17460, 'start': 16304, 'socializing': 15904, 'with': 18944, 'machines': 10469, 'why': 18869, 'wouldn': 19045, 'they': 17285, 'be': 2189, 'all': 1216, 'road': 14786, 'signs': 15637, 'pointed': 13030, 'towards': 17560, 'insane': 9055, 'success': 16636, 'at': 1811, 'mobile': 11129, 'world': 19025, 'congress': 3980, '2017': 288, 'main': 10499, 'headliners': 8138, 'conference': 3937, 'organizers': 12143, 'cited': 3372, 'an': 1345, 'overwhelming': 12299, 'acceptance': 798, 'event': 63

### Transformace dotazu a výpočet relevance

In [37]:
def cos_sim(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [38]:
query = 'cousera vs udacity machine learning'
query_tfidf = vectorizer.transform([query])

print(query_tfidf.shape)

query_vector = calculate_doc_vector(doc_id=None, 
                                      tfidf_matrix=query_tfidf, 
                                      word_to_index=word_to_index, 
                                      glove=glove)

(1, 20396)


In [39]:
scores = []    

for doc_id in range(len(texts)):
    doc_vector = calculate_doc_vector(doc_id=doc_id, 
                                    tfidf_matrix=tfidf_matrix, 
                                    word_to_index=word_to_index, 
                                    glove=glove)
    
    score = cos_sim(query_vector, doc_vector)
    scores.append(score)

In [ ]:
data = {
        'Title': titles,
        'Text': texts,
        'Score': scores
    }
df = pd.DataFrame(data)
df_sorted = df.sort_values(by='Score', ascending=False)
print(df_sorted)

## Bonus - Našeptávání

Bonusem dnešního cvičení je našeptávání pomocí rekurentních neuronových sítí. Úkolem je vytvořit jednoduchou rekurentní neuronovou síť, která bude generovat text (character-level přístup). 

Optimální je začít po dokončení cvičení k předmětu ANS, kde se tato úloha řeší. 

Dataset pro učení vaší neuronové sítě naleznete na stránkách [Yahoo research](https://webscope.sandbox.yahoo.com/catalog.php?datatype=l&guccounter=1), lze využít např. i větší [Kaggle dataset](https://www.kaggle.com/c/yandex-personalized-web-search-challenge/data) nebo vyhledat další dataset na [Google DatasetSearch](https://datasetsearch.research.google.com/).

Vstupem bude rozepsaný dotaz a výstupem by měly být alespoň 3 dokončené dotazy.